In [183]:
# -*- coding: utf-8 -*-
import numpy as np
import urllib.request, json, re
import pprint as pp
import tensorflow as tf
import re

from tensorflow.python.framework import ops
import matplotlib.pyplot as plt
import time
import datetime


import random
np.random.seed(1)

# Klassformat för lagring och serialisering
    

In [184]:
class DocList:
    data = []
    def __init__(self):
        self.name='MotionList'
        self.data = []
    def reprJSON(self):
        return dict(name=self.name, data=self.data) 
    
class Motion:
    def __init__(self):
        self.text=''
        self.title=''
        self.subtype=''
        self.meta = {}
    def reprJSON(self):
        return dict(text=self.text, subtype=self.subtype, title=self.title, meta=self.meta) 
    
class DocSerializer(json.JSONEncoder):
    def default(self, obj):
        if hasattr(obj,'reprJSON'):
            return obj.reprJSON()
        else:
            return json.JSONEncoder.default(self, obj)



## Import och skapande av datafil
denna koden ska bara köras en gång vid inläsning av data från riksdagen

In [200]:

def import_web_data(datasize=3):
    pages = int(np.floor(datasize/201))+1
    #http://data.riksdagen.se/dokumentlista/?doktyp=mot
    #http://data.riksdagen.se/dokument/H502225.text (exempeldok.)
    #"http://data.riksdagen.se/anforandelista/?rm=&anftyp=Nej&d=&ts=&parti=s&iid=0218878014918&sz="+str(datasize)+"&utformat=json"
    #http://data.riksdagen.se/anforandelista/?rm=&anftyp=&d=&ts=&parti=&iid=&sz=10&utformat=xml
    motioner = DocList()
    for page in range(pages):
        with urllib.request.urlopen("http://data.riksdagen.se/dokumentlista/?doktyp=mot&sz="+str(datasize)+"&utformat=json&p="+str(page+1)) as url:
            rawdata = json.loads(url.read().decode())
            motionslista = rawdata['dokumentlista']['dokument']
            for meta in motionslista:
                if(not not meta):
                    motion = Motion()
                    with urllib.request.urlopen('http:' + meta['dokument_url_text']) as text:
                        motion.text = re.sub('\n','',text.read().decode())
                        #motion.meta = meta (save space appr 30%)
                        motion.title = meta['titel']
                        motion.subtype = meta['subtyp']

                    motioner.data.append(motion)

     
    toJson = json.dumps(motioner.reprJSON(), cls=DocSerializer,  sort_keys=True, indent=4)
    with open('motioner_big.txt','w',encoding='utf-8') as f:
        
        f.write(toJson) #json.dumps(motions.reprJSON()))#json.dumps(motions, sort_keys=True, indent=4))
        
    return toJson, motioner

#datasize är antal dokument vi behöver förmodligen ett hundratal iaf. (nedan är bortkommenterad om man har fil)
data_ex, motionList = import_web_data(datasize=2000)
print("totalt antal dokument: " + str(len(motionList.data)))



totalt antal dokument: 2000


In [202]:
def file_to_json(filename):
    with open(filename,encoding="utf-8") as f:
        data = json.loads(f.read())
        return data

def json_to_vec(jsondata):
    docs = []
    words = []
    for sample in jsondata:
        #remove non letters
        cleaned = sample["text"].lower()
        cleaned = re.sub('[\r\r]', '', cleaned) #måste vara såhär (då ord kan vara delade varsomhelst med \r\r)
        cleaned = re.sub('[\n|\)|\(]', ' ', cleaned)
        PERMITTED_CHARS = "abcdefghijklmnopqrstuvwxyzåäöABCDEFGHIJKLMNOPQRSTUVWXYZÅÄÖ "

        cleaned = "".join(c for c in cleaned if c in PERMITTED_CHARS)
        #cleaned = re.sub(r'^\w', '', sample["text"].lower())
        #cleaned =  sample["text"].lower()
        #remove space
        cleaned =' '.join( [x for x in re.split(r'[\u200b\s]+', cleaned, flags=re.UNICODE) if x != ''])
        words += cleaned.lower().split()
        docs.append(cleaned)

    #print(speech)
    #data = tf.compat.as_str(speech).split()
    orig_dic = dict(list(zip(words,words)))
    return docs, orig_dic

In [203]:
jsondata = file_to_json("motioner.txt")

docs, orig_dic = json_to_vec(jsondata['data'])

lista = list(set(orig_dic.keys()))
lista.sort()
print(lista[6300:6310])

['hemavdrag', 'hembesök', 'hembygd', 'hembygden', 'hemelektronik', 'hemifrån', 'hemkunskap', 'hemkunskapsämnet', 'hemland', 'hemlandet']


#  läs från fil

In [204]:
jsondata = file_to_json("motioner.txt")

docs, orig_dic = json_to_vec(jsondata['data'])

print(docs[3])


motion till riksdagenav oscar sjöstedt och dennis dioukarev båda sd med anledning av prop bättre förutsättningar för fondsparande och hållbara valförslag till riksdagsbeslutriksdagen avslår förslaget i de delar som avser krav på att tillhandahållande av hållbarhetsinformation regleras i lag och därtill knutna bestämmelser i förslagen till lag om värdepappersfonder och lag om förvaltare av alternativa investeringsfonder avsnitt och motiveringregeringens föreslår att genom lag reglera att fonder ska lämna information om förvaltning med avseende på olika hållbarhetsaspekter så som miljö sociala förhållanden personal respekt för mänskliga rättigheter och korruption vidare ska samtliga fonder vare sig de beaktar hållbarhetsaspekter eller ej också åläggas med informationskrav om att rapportera detta det finns idag flera internationellt erkända alternativ för hållbarhetsinformation fn initiativet principles for responsible investments pri som förtydligar redovisning inom hållbarhet med kapita

#  Lemmatisering av alla ord
Här måste vi gå igenom alla ord och rensa upp för att sedan ersätta ord i texten med en mindre ordrymd

In [308]:
def indexOf(text, elem):
        try:
            thing_index = text.index(elem)
            return thing_index
        except ValueError:
            return -1

        
#skapar en dictionary över gammalt till nytt ord så man kan köra find-replace med den senare.
def lemmatize_and_clean(orig_dic, sensitivity=.5):        
    # a jph production yeahh
    
    # unika ord (set)
    myset = set(list(orig_dic.keys()))
    # till lista igen
    all_words = list(myset)
    #sortera bokstav
    all_words.sort(reverse=False)
    
 
    orig = all_words
    newlist = []
    #regexp = re.compile('(.?[0-9]+)|^[/&%\[=\–]') #rensa tecken
    for i in range(len(orig)):
        #if(regexp.match(orig[i])):
        #    newlist.append('')
        #else:
            fidx = 0
            if(len(newlist)>0):
                suffix=0
                if(len(orig[i])>7):
                    suffix = 2
                fidx = (indexOf(orig[i],newlist[i-1][:len(orig[i])-suffix])+1)
            #kolla om längd på aktuellt ersättningsord (newlist[i-1]) är över dellängd av nuvarande ord (eg .6 = 60%)
            #ifall denna skillnad blir för stor byt ersättningsord
            if(fidx != 0 and ((1.0*len(newlist[i-1])/len(orig[i]) >=sensitivity) and len(newlist[i-1]) > 4)):
                newlist.append(newlist[i-1])
            else:
                newlist.append(orig[i])
    
    
    return dict(list(zip(orig,newlist)))


orig_dic  = lemmatize_and_clean(orig_dic,sensitivity=.4)

print("from "+str(len(orig_dic))+" to: "+str(len(set(list(orig_dic.values())))))

#print(orig_dic)


from 19479 to: 11476


TODO: använd wdict på alla textdokument för att ersätta affärens, affärsmodell, affärsrunda... med affär (enl ovan)

# Stop words
ta bort ord som inte säger något som prepositioner mm. Min naiva approach är att kolla upp de vanligaste orden och ta bort dem. Tror dock man ska kolla (http://stevenloria.com/finding-important-words-in-a-document-using-tf-idf/)
St (pip install stop-words yay)

In [9]:
from stop_words import get_stop_words

def remove_stop_words(dic):
    stop_words = get_stop_words('sv')
    return {k:('' if (dic[k] in stop_words) else dic[k]) for k in dic}
    

In [205]:
jsondata = file_to_json("motioner_big.txt")
texts, orig_dic = json_to_vec(jsondata['data'])

stop_dic = remove_stop_words(orig_dic)
print("remove Stopwords from "+str(len(orig_dic))+" to: "+str(len(set(list(stop_dic.values())))))

lemma_dic = lemmatize_and_clean(stop_dic,sensitivity=.4)
print("remove suffix from "+str(len(set(list(stop_dic.values()))))+" to: "+str(len(set(list(lemma_dic.values())))))

remove Stopwords from 90117 to: 90005
remove suffix from 90005 to: 50527


# Preprocessing

In [206]:
def build_dict(orig_dic):
    before = str(len(orig_dic.keys()))
    dic = remove_stop_words(orig_dic)
    dic = lemmatize_and_clean(dic,sensitivity=.4)
    print("Limit words from "+before+" to: "+str(len(set(list(dic.values()))))+"\n")
    return dic


##SLOW AS FUCK.
def preprocess(texts,dic):
    lst = [re.escape(key) for key in dic.keys()]
    pattern = re.compile(r'\b(' + '|'.join(lst) + r')\b')
    cleaned = []
    for text in texts:
        cleaned.append(pattern.sub(lambda x: dic[x.group()], text))
    return cleaned

In [257]:
def batch_clean(file_in, file_out):
    jsondata = file_to_json(file_in)
    texts, orig_dic = json_to_vec(jsondata['data'])
    dic = build_dict(orig_dic)
    texts = preprocess(texts,dic)
    with open(file_out,'w',encoding='utf-8') as f:
        f.write(json.dumps(texts, sort_keys=True, indent=4))
        


In [ ]:
batch_clean('motioner_big.txt','motioner_cleaned.txt')

## Test

In [ ]:
#ta ut alla dokument
jsondata = file_to_json("motioner_big.txt")
texts, orig_dic = json_to_vec(jsondata['data'])

dic = build_dict(orig_dic)
#print(dic)
ts = texts[2:3]
print(ts,len(ts[0]))
texts = preprocess(ts,dic)

print("\n",texts,len(texts[0]))


# Nästa steg - Bag of words
skapa vektor av alla ord. Eg bara köra list(set(orig_dic.values())) 
Skapa np.arrays där varje dokument har en siffra i varje position i ovan vektor.
Detta blir då ett training example eller ett x i X.

# NMF

för att sedan få matrisrepresentation som går att köra NonNegative Matrix Factorization på.

http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html

In [212]:
def sample_to_vec(sample, wi):
    vocabular = wi
    sample_words = np.array(sample.split(" "))
    vec = np.array([wi[x] for x in sample_words])
    one_hot = np.zeros([len(vec), len(vocabular)])
    one_hot[np.arange(len(vec)),vec] = 1
    one_hot = np.sum(one_hot,axis=0)
    return vec,one_hot

#generera index - ord till id och id till ord
def gen_idx(texts):
    all_unique_words = list(set((''.join(texts)).split(" ")))
    word_to_idx = {i:w for w,i in enumerate(all_unique_words)}
    idx_to_word = {i:w for i,w in enumerate(all_unique_words)}   
    return word_to_idx, idx_to_word


# samples_to_train:
# [sampletxt, sampletxt ...] => [n x m], [word->index], [index->word]
# där n = ordlistas längd (features) och m = antal dokument
def samples_to_train(samples):
    vocabulary, iw = gen_idx(samples)
    sample_vector = []
    for sample in samples:
        _, x = sample_to_vec(sample, vocabulary)
        sample_vector.append(x)
    return np.matrix(sample_vector).reshape((len(samples),len(vocabulary))).T, vocabulary, iw


# går inte att köra för preprocess är slött
Detta steg är viktigt för att få bort massa skräp.

In [258]:
batch_clean('motioner_big.txt','motioner_cleaned.txt')

Limit words from 90117 to: 50527



In [291]:
jsondata = file_to_json("motioner.txt")
texts, orig_dic = json_to_vec(jsondata['data'])

X_samp, wi, iw = samples_to_train(texts)


In [292]:
X = np.array([[1, 1], [2, 1], [3, 1.2], [4, 1], [5, 0.8], [6, 1]])
from sklearn.decomposition import NMF
model = NMF(n_components=3, init='random', random_state=0)
W = model.fit_transform(X)
H = model.components_
print(H)
print(W)

[[ 1.66585801  0.08635763]
 [ 0.73400202  2.06222242]
 [ 1.86096512  0.31755657]]
[[ 0.38534454  0.4675586   0.00799441]
 [ 0.          0.34002261  0.94060871]
 [ 1.34627306  0.49279364  0.21257051]
 [ 1.49619407  0.31673295  0.68516976]
 [ 0.26846283  0.          2.44645467]
 [ 2.44632733  0.2376055   0.94057293]]


In [296]:
topics = 4
model = NMF(n_components=topics, init='random', random_state=0)
W = model.fit_transform(X_samp.T)
H = model.components_

In [297]:
print(W.shape) #dokument och topic
print(H.shape) #topics och ord

(200, 4)
(4, 19487)


In [305]:
#topic 1 - detta verkar inte funka riktigt då jag inte tar bort ord som förekommer i alla dok.
topic = 1
t1 = dict(zip(H[topic],range(len(H[topic]))))
t1 = sorted(t1.items(),reverse=True)
for w,idx in t1[:20]:
    print(iw[idx])


att
i
som
och
för
det
av
en
om
till
är
ett
detta
den
sig
på
med
ska
inte
har


In [309]:
#SciPy färdiga NMF med tfidf (samma som ovan, fast man slipper slå upp allt mot arrayer samt rensa ord.)
topics = 4
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                   max_features=3000,
                                   stop_words=get_stop_words('sv'))
X_samp = tfidf_vectorizer.fit_transform(texts)


nmf = NMF(n_components=4, random_state=1,shuffle=True,
          alpha=.1, l1_ratio=.7).fit(X_samp)


vocabulary = tfidf_vectorizer.get_feature_names()
#print(vocabulary)
print_top_words(nmf, vocabulary, 10)
print("samples, max words/doc:" + str(X_samp.shape))

Topic #0: ska bör sverige regeringenriksdagen bakom ställer motionen anförs tillkännager regeringen
Topic #1: miljoner kronor beräknas minska år föreslås följd anslaget belopp centerpartiets
Topic #2: riksdagenmotionen utgår historiska hindras hindrar hindra hinder hen hemvist hemmet
Topic #3: skalin johnny sd tillkännajohnny bör anförs ges motionen sista regeringen

samples, max words/doc:(200, 3000)
